In [ ]:
!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb, Discover, Genre
from datetime import datetime, timedelta
import boto3
import json
from botocore.exceptions import ClientError
import logging
import os

# Criar uma sessão do boto3

ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
SECRET_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
SESSION_TOKEN = os.environ['AWS_SESSION_TOKEN']
API_KEY = os.environ["API_KEY"]

session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN
)

def lambda_handler(event, context):
    tmdb = TMDb()
    tmdb.api_key = API_KEY
    tmdb.language = 'pt-BR'
    
    # metodos TMDB para achar os filmes
    discover = Discover()
    genre = Genre()
    
    # Obtenha a lista de gêneros de filmes
    genres = genre.movie_list()
    
    # Criando um dicionário para mapear IDs de gênero para nomes de gênero
    genre_dict = {g['id']: g['name'] for g in genres}
    
    # Parametros para pegar as datas do intervalo
    params = {
        'with_genres': 27 # ID do gênero de terror
    }
    
    # count contar a quantidade de filmes achados
    # escolher a quantidade de paginas da API
    count = 0
    movies_data = []
    
    for page in range(1, 101):
        params['page'] = page
        movies = discover.discover_movies(params)
        count += len(movies)
    
        for movie in movies:
            print(movie)
            movie_data = movie.__dict__.copy()
            movie_data['genres'] = [genre_dict[g] for g in movie['genre_ids']]
            movies_data.append(movie_data)
    
    # Convertendo a lista de dados dos filmes em formato JSON
    movies_json = json.dumps(movies_data, ensure_ascii=False)

    
    # Conexão com o S3
    s3 = session.client('s3')
    
    # O caminho completo no bucket S3
    bucket_name = 'data-lake-heliton'
    current_date = datetime.now()
    year = current_date.strftime('%Y')
    month = current_date.strftime('%m')
    day = current_date.strftime('%d')
    object_key = f'RAW/TMDB/JSON/{year}/{month}/{day}/filmes_popular.json'
    
    # Envio do JSON para o bucket no S3
    s3.put_object(
        Body=movies_json,
        Bucket=bucket_name,
        Key=object_key
    )